In [1]:
import os
from os import path
import pylangacq

In [10]:
class Paths:
  path = '/vol/tensusers3/thijsdejong/2324-asr'
  transcriptions = f'{path}/datasets/addresso/test/transcription'
  text_participants = f'{path}/datasets/addresso/test/text_participants'
  text_both = f'{path}/datasets/addresso/test/text_both'

In [11]:
def path_to_text(path, full_transcript=False):
    chat = pylangacq.read_chat(path)
    utterances = [utterance for utterance in chat.utterances()]
    tokens = [(token, utterance.participant) for utterance in utterances for token in utterance.tokens]
    words = [(token.word, participant) for (token, participant) in tokens]

    if not full_transcript:
      words = list(filter(lambda w: w[1] == 'PAR', words))
    text = ''
    for i, word_par in enumerate(words):
      word = word_par[0]
      par = word_par[1]
      if word in ['POSTCLITIC']:
        continue
      if word in ['.', '?', '!']:
        text += f'{word}\n'
      else:
        if i == 0 or words[i-1][0] in ['.', '?', '!']:
          if full_transcript:
            text += f'{par}: '
          text += word.capitalize()
        else:
          text += word
        if i < len(words) - 1 and words[i+1][0] not in ['.', ',', '?', '!']:
          text += ' '
    return text

In [13]:
print(path_to_text(path.join(Paths.transcriptions, 'S160.cha'), full_transcript=True))

PAR: This boy is about to fall off of the stool.
PAR: The mother is washing dishes and the water's spilling over on the kitchen floor.
PAR: The wind is blowing the curtains.
PAR: The little girl is laughing at her brother who's taking a cookie out_of the cookie jar.
PAR: Think that's it.
INV: Okay.



In [14]:
for file in os.listdir(Paths.transcriptions):
  if file.endswith('.cha'):
    ID = file.split('.')[0]
    print(f'Processing {ID}...')

    text_participants = path_to_text(path.join(Paths.transcriptions, file), full_transcript=False)
    text_both = path_to_text(path.join(Paths.transcriptions, file), full_transcript=True)

    if not os.path.exists(Paths.text_participants):
      os.makedirs(Paths.text_participants)
    with open(path.join(Paths.text_participants, f'{ID}.txt'), 'w') as f:
      f.write(text_participants)

    if not os.path.exists(Paths.text_both):
      os.makedirs(Paths.text_both)
    with open(path.join(Paths.text_both, f'{ID}.txt'), 'w') as f:
      f.write(text_both)

Processing S165...
Processing S166...
Processing S167...
Processing S168...
Processing S169...
Processing S170...
Processing S171...
Processing S172...
Processing S173...
Processing S174...
Processing S175...
Processing S176...
Processing S177...
Processing S178...
Processing S179...
Processing S180...
Processing S181...
Processing S182...
Processing S183...
Processing S184...
Processing S185...
Processing S186...
Processing S187...
Processing S188...
Processing S189...
Processing S190...
Processing S191...
Processing S192...
Processing S193...
Processing S194...
Processing S195...
Processing S196...
Processing S197...
Processing S198...
Processing S199...
Processing S200...
Processing S201...
Processing S202...
Processing S203...
Processing S204...
Processing S205...
Processing S206...
Processing S207...
Processing S164...
Processing S160...
Processing S161...
Processing S162...
Processing S163...
